In [1]:
import pandas as pd
from statistics import mode
import json, os, pickle

In [2]:
from datetime import datetime
input_ = {
    "model_id": ["20231025000000", "20231026000000"],
    "upload_time":  datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "machine_id": "XXXX",
    "path": r"C:\Users\tzuli\Documents\python\hosong\data\predict"
}

root = os.getcwd()


model_id = input_["model_id"]
upload_time = input_["upload_time"]
machine_id = input_["machine_id"]


# 取得input
input_json = os.path.join(input_["path"], "input.json")
output_json = os.path.join(input_["path"], "output.json")
with open(input_json, encoding = "utf-8") as f:
    input_ = json.load(f)


# 取得model位置
ng4_5_detail = os.path.join(root, "data", "train", model_id[0], "ng4_5", "model")
ng1_3_detail = os.path.join(root, "data", "train", model_id[1], "ng1_3", "model")


# NG歷史紀錄
pred_path = os.path.join(root, "data", "predict")        
os.makedirs(pred_path, exist_ok = True)
ng1_3_history_path = os.path.join(pred_path, "NG1_3.json")

if os.path.exists(ng1_3_history_path): # 如果檔案存在，則讀取檔案內容    
    with open(ng1_3_history_path, encoding="utf-8") as f:
        ng1_3_history = json.load(f)
else: # 如果檔案不存在，則創建一個新的dict
    ng1_3_history = {}


# 取得config
config_path = os.path.join(root, "prog", "config.json")
with open(config_path, encoding = "utf-8") as f:
    config = json.load(f)
model_cols = config["model_cols"] # set columns for each models
directions = config["directions"]

In [3]:
df = pd.DataFrame(input_, index = [0])
df

,直徑(1)\n圓(A0.左邊)_NV,直徑(1)\n圓(A0.左邊)_AV,直徑(1)\n圓(A0.左邊)_DV,直徑(1)\n圓(A0.左邊)_UT,直徑(1)\n圓(A0.左邊)_LT,直徑(1)\n圓(A0.左邊)_ER,Y距離(7)\n距離(A0.左邊)_NV,Y距離(7)\n距離(A0.左邊)_AV,Y距離(7)\n距離(A0.左邊)_DV,Y距離(7)\n距離(A0.左邊)_UT,...,Y距離(6)\n距離(A180.左邊)(A180.右邊)_DV,Y距離(6)\n距離(A180.左邊)(A180.右邊)_UT,Y距離(6)\n距離(A180.左邊)(A180.右邊)_LT,Y距離(6)\n距離(A180.左邊)(A180.右邊)_ER,XY距離(1)\n距離_NV,XY距離(1)\n距離_AV,XY距離(1)\n距離_DV,XY距離(1)\n距離_UT,XY距離(1)\n距離_LT,XY距離(1)\n距離_ER
0,98.0,98.01,0.01,0.022,0.0,0.0,100.0,99.998,-0.002,0.01,...,0.009,0.019,0.0,0.0,240.3,240.312,0.012,0.05,-0.05,0.0


In [4]:
counts = len(set([i.split("_")[0] for i in input_.keys() if i != "time"])) # 共多少點

In [5]:
# check NG or OK
target_ER = [model_col["primary"][0] + "_ER" for model_col in model_cols.values()]
target_value = df.loc[0, target_ER]
ng_col = target_value[target_value != 0].index.str.replace('_ER', '').tolist()

ng_flag = (target_value != 0).any()
print(f"ng_col = {ng_col}")
print(f"ng_flag = {ng_flag}")

ng_col = []
ng_flag = False


In [6]:
# OK
if not ng_flag:
    result = {    
        "status":   "success",
        "upload_time": upload_time,
        "model_id": model_id,
        "result": 0,
        "counts": counts
        }
    print(result)

{'status': 'success', 'upload_time': '2023-11-09 10:59:20', 'model_id': ['20231025000000', '20231026000000'], 'result': 0, 'counts': 15}


In [7]:
# calculate total ER by each side
left_ER_col = [col for col in df.columns if ("ER" in col) and ("左邊" in col) and ("右邊" not in col)]
right_ER_col = [col for col in df.columns if ("ER" in col) and ("右邊" in col) and ("左邊" not in col)]
left  = df[left_ER_col].sum(axis = 1)
right = df[right_ER_col].sum(axis = 1)

In [8]:
# choose the adjustment method
error = abs(right - left)[0]
print(f"error = {error}")
ng4_5_flag = (error <= 0.001)
ng4_5_flag

error = 0.0


True

In [9]:
# predict
pred = {}
if ng4_5_flag:
    print("NG4_5 model.")
    model = pickle.load(open(os.path.join(ng4_5_detail, "model.pkl"), "rb"))
    features = pickle.load(open(os.path.join(ng4_5_detail, "features.pkl"), "rb"))
    X_test = df[features]
    test_pred = model.predict(X_test)[0]
    pred.update({
        "model": "ng4_5",
        "adjust": test_pred
    })

else:
    print("NG1_3 model.")
    pred.update({
        "model": "ng1_3",
        "adjust": {},
    })
    for i, model_col in enumerate(model_cols.values(), start = 1):
        primary = model_col["primary"][0]
        if primary in ng_col:
            model = pickle.load(open(os.path.join(ng1_3_detail, f"model{i}.pkl"), "rb"))        
            scaler = pickle.load(open(os.path.join(ng1_3_detail, f"scaler{i}.pkl"), "rb"))
            features = pickle.load(open(os.path.join(ng1_3_detail, f"features{i}.pkl"), "rb"))

            X_test = df[features]
            X_test = scaler.transform(X_test)
            test_pred = model.predict(X_test).argmax(axis = 1)[0]
            pred["adjust"][primary] = int(test_pred + 1)  # NG1: -0.001, NG2: 0, NG3: +0.001
        else:            
            pred["adjust"][primary] = 2 # 沒NG補2
pred

NG4_5 model.


{'model': 'ng4_5', 'adjust': 1}

In [10]:
# save NG hisitory
if pred["model"] == "ng1_3":
    if (machine_id in ng1_3_history):
        ng1_3_history[machine_id][upload_time] = pred["adjust"]
    else:
        ng1_3_history[machine_id] = {upload_time: pred["adjust"]}
        
    with open(ng1_3_history_path, 'w', encoding = 'utf-8') as file:
        json.dump(ng1_3_history, file, indent = 4, ensure_ascii = False)

In [11]:
# bulid compensate
mode_pred = []
if pred["model"] == "ng1_3":
    compensate = {}
    for primary, adjust in pred["adjust"].items():
        if adjust == 2: # NG1: -0.001, NG2: 0, NG3: +0.001，2不須補償
            continue
        
        point = primary.split("距離")[0] # point

        # direction and no
        if "A0" in primary:
            direction = directions["A0"]["name"]
            no = directions["A0"]["no"]
        elif "A180" in primary:
            direction = directions["A180"]["name"]
            no = directions["A180"]["no"]
        else:
            direction = directions["A0+A180"]["name"]
            no = directions["A0+A180"]["no"]

        # side
        if ("右邊" in primary) and ("左邊" in primary):
            side = "left_right"
        elif "左邊" in primary:
            side = "left"
        elif "右邊" in primary:
            side = "right"
        else:
            side = "left_right"

        adjust1 = -0.001 if (adjust == 1) else 0.001 # NG1: -0.001, NG2: 0, NG3: +0.001

        if (direction in compensate) and (side in compensate[direction]):
            compensate[direction][side] = {"no": no, point: adjust1}
        else:
            compensate[direction] = {
                side:{
                    "no": no,
                    point: adjust1
                }
            }
        
        mode_pred.append(adjust) # 最後result從所有adjust中不為NG2的取眾數
    
    # get result
    if "P15" in compensate:
        result = 3
    elif "P14" in compensate:
        result = 2
    elif "P13" in compensate:
        result = 1
    else:
        result = 0
else:
    result, compensate = (4, "NC_FIX_1") if pred["adjust"] == 0 else (5, "NC_FIX_2")

In [12]:
# fix A180+A0 compensate, split XY to X & Y 
if directions["A0+A180"]["name"] in compensate:
    XY = compensate[directions["A0+A180"]["name"]]["left_right"].pop("XY")
    compensate[directions["A0+A180"]["name"]]["left_right"].update({
        "X": XY,
        "Y": XY
    })

In [13]:
result = {    
    "status":   "success",
    "upload_time": upload_time,
    "model_id": model_id,
    "result": result,
    "counts": counts
    }
if compensate:
    result["compensate"] = compensate
result

{'status': 'success',
 'upload_time': '2023-11-09 10:59:20',
 'model_id': ['20231025000000', '20231026000000'],
 'result': 5,
 'counts': 15,
 'compensate': 'NC_FIX_2'}

In [14]:
output_json

'C:\\Users\\tzuli\\Documents\\python\\hosong\\data\\predict\\output.json'

In [15]:
with open(output_json, 'w', encoding = 'utf-8') as file:
    json.dump(result, file, indent = 4, ensure_ascii = False)